- ref. 

In [145]:
# load data
import pandas as pd, numpy as np
pd.set_option("display.width", 80)
pd.set_option("display.max_rows", 20)
import seaborn as sns, matplotlib.pyplot as plt
import multiprocessing as mp
n_cpus = mp.cpu_count()

# train_df = pd.read_csv("./train.csv")   # 42000
train_df = pd.read_csv("./train.csv", nrows=20000)
test_df = pd.read_csv("./test.csv")   # 28000
train_num = train_df.shape[0]
test_num = test_df.shape[0]
train_x = train_df.drop(['label'], axis=1)
train_y = train_df["label"]
test_x = test_df
all_df = pd.concat((train_x, test_df))

dfd = {'train': train_df, 'test': test_df, 'all': all_df}
for name,df in dfd.items():
    print(name, df.shape)
    print(df.head())

train (20000, 785)
   label  pixel0  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  \
0      1       0       0       0       0       0       0       0       0   
1      0       0       0       0       0       0       0       0       0   
2      1       0       0       0       0       0       0       0       0   
3      4       0       0       0       0       0       0       0       0   
4      0       0       0       0       0       0       0       0       0   

   pixel8    ...     pixel774  pixel775  pixel776  pixel777  pixel778  \
0       0    ...            0         0         0         0         0   
1       0    ...            0         0         0         0         0   
2       0    ...            0         0         0         0         0   
3       0    ...            0         0         0         0         0   
4       0    ...            0         0         0         0         0   

   pixel779  pixel780  pixel781  pixel782  pixel783  
0         0         0         0

In [146]:
# check data
for name,df in dfd.items():
    print(name)
    print(df.info())
    #print(df.describe())   # doesn't show a lot of information

train
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 119.8 MB
None
test
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 167.5 MB
None
all
<class 'pandas.core.frame.DataFrame'>
Int64Index: 48000 entries, 0 to 27999
Columns: 784 entries, pixel0 to pixel783
dtypes: int64(784)
memory usage: 287.5 MB
None


In [147]:
# check null
for name,df in dfd.items():
    print(name)
    isnull_ratio = df.isnull().sum()/df.isnull().count()
    print(isnull_ratio[isnull_ratio != 0].sort_values(ascending=False))

Series([], dtype: float64)


Series([], dtype: float64)
all


Series([], dtype: float64)
test


train


In [148]:
# check correlation (takes time)
# -> pixel436 has high correlation with label.. so what?
# correlations = train_df.astype(float).corr()['label'].sort_values(ascending=False)
# print(correlations.head())
# print(correlations.tail())

In [149]:
# check skewness (takes time)
# -> pixel367 has high skewness, but log/boxcox transform doesn't help
# from scipy.stats import skew
# skewness = all_df.apply(lambda x: skew(x)).sort_values(ascending=False)
# print(skewness.head())
# print(skewness.tail())
# all_df['pixel367'].hist(bins=50)
# plt.show()

In [150]:
# grid search
from sklearn.linear_model import LogisticRegression, Perceptron, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
train_x = all_df[:train_num]
test_x = all_df[train_num:]

models = []
C_list, gamma_list, degree_list = np.logspace(-2, 0, 3), np.logspace(-3, -1, 3), np.linspace(1, 3, 3)
grid = [
    {'C': C_list, 'kernel': ['poly'], 'gamma': gamma_list, 'degree': degree_list},
    # {'C': C_list, 'kernel': ['sigmoid'], 'gamma': gamma_list},
    # {'C': C_list, 'kernel': ['linear']},
    # {'C': C_list, 'kernel': ['rbf'], 'gamma': gamma_list},
    ]
models.append(('svm', GridSearchCV(SVC(), grid, cv=3, n_jobs=n_cpus)))
grid = [{'n_estimators': [1000], 'max_depth': [6,7,8], 'min_samples_leaf': [2,3], 'max_features' : ['sqrt']}]
models.append(('random_forest', GridSearchCV(RandomForestClassifier(), grid, cv=3, n_jobs=n_cpus)))
models.append(('extra_trees', GridSearchCV(ExtraTreesClassifier(), grid, cv=3, n_jobs=n_cpus)))
grid = {'n_estimators': [1000], 'learning_rate' : [0.25, 0.5, 0.75]}
models.append(('ada_boost', GridSearchCV(AdaBoostClassifier(), grid, cv=3, n_jobs=n_cpus)))
grid = {'n_estimators': [1000], 'max_depth': [6,7,8], 'min_samples_leaf': [2,3]}
models.append(('gradient_boosting', GridSearchCV(GradientBoostingClassifier(), grid, cv=3, n_jobs=n_cpus)))
grid = {'max_iter': [1000], 'tol': [0.001]}
models.append(('perceptron', GridSearchCV(Perceptron(max_iter=1000, tol=0.001), grid, cv=3, n_jobs=n_cpus)))
models.append(('sgd_classifier', GridSearchCV(SGDClassifier(max_iter=1000, tol=0.001), grid, cv=3, n_jobs=n_cpus)))
grid = {'n_neighbors': [2,3,4]}
models.append(('k-nearest_neighbors', GridSearchCV(KNeighborsClassifier(), grid, cv=3, n_jobs=n_cpus)))
grid = {}
models.append(('logistic_regression', GridSearchCV(LogisticRegression(), grid, cv=3, n_jobs=n_cpus)))
models.append(('gaussian_naive bayes', GridSearchCV(GaussianNB(), grid, cv=3, n_jobs=n_cpus)))
models.append(('linear_svc', GridSearchCV(LinearSVC(), grid, cv=3, n_jobs=n_cpus)))
models.append(('decision_tree', GridSearchCV(DecisionTreeClassifier(), grid, cv=3, n_jobs=n_cpus)))

params = {}
for name,model in models:
    model.fit(train_x, train_y)
    print(name, model.best_score_, model.best_params_)
    params[name] = model.best_params_
    # results = model.cv_results_
    # for mean, std, params in zip(results['mean_test_score'], results['std_test_score'], results['params']):
    #     print("{:0.3f} (+/-{:0.03f} for {}".format(mean, std, params))

models = [
       ('svm', SVC(**params['svm'])),
       ('random_forest', RandomForestClassifier(**params['random_forest'])),
       ('extra_trees', ExtraTreesClassifier(**params['extra_trees'])),
       ('ada_boost', AdaBoostClassifier(**params['ada_boost'])),
       ('gradient_boosting', GradientBoostingClassifier(**params['gradient_boosting'])),
       ('perceptron', Perceptron(**params['perceptron'])),
       ('sgd_classifier', SGDClassifier(**params['sgd_classifier'])),
       ('k-nearest_neighbors', KNeighborsClassifier(**params['k-nearest_neighbors'])),
       ('logistic_regression', LogisticRegression(**params['logistic_regression'])),
       ('gaussian_naive bayes', GaussianNB(**params['gaussian_naive bayes'])),
       ('linear_svc', LinearSVC(**params['linear_svc'])),
       ('decision_tree', DecisionTreeClassifier(**params['decision_tree'])),
       ]

decision_tree 0.8189 {}


linear_svc 0.8538 {}


gaussian_naive bayes 0.5606 {}


logistic_regression 0.8562 {}


k-nearest_neighbors 0.9543 {'n_neighbors': 3}


...

gradient_boosting 0.9545 {'max_depth': 6, 'min_samples_leaf': 3, 'n_estimators': 1000}


ada_boost 0.71145 {'learning_rate': 0.25, 'n_estimators': 1000}


extra_trees 0.9146 {'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 1000}


random_forest 0.92315 {'max_depth': 8, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'n_estimators': 1000}


svm 0.9645 {'C': 0.01, 'degree': 2.0, 'gamma': 0.001, 'kernel': 'poly'}


In [151]:
# cross validation
from sklearn.model_selection import KFold, cross_validate
from multiprocessing import Manager

k_fold = KFold(n_splits=3)
with Manager() as manager:
    results = manager.list()
    def validate_one_model(name, model):
         # Multiprocessing-backed parallel loops cannot be nested
         scores = cross_validate(model, train_x, train_y, cv=k_fold)
         result = {}
         result['model'] = name
         result['test_mean'] = np.mean(scores['test_score'])
         result['test_std'] = np.std(scores['test_score'])
         result['train_mean'] = np.mean(scores['train_score'])
         result['train_std'] = np.std(scores['train_score'])
         results.append(result)
    Parallel(n_jobs=n_cpus)(delayed(validate_one_model)(name, model) for name,model in models)
    results_merged = {'model': [], 'test_mean': [], 'test_std': [], 'train_mean': [], 'train_std': []}
    for result in results:
        for k,v in result.items():
            results_merged[k].append(v)
    print(pd.DataFrame(results_merged).sort_values(by='test_mean', ascending=False))

                   model  test_mean  test_std  train_mean  train_std
7                    svm   0.963950  0.002717    1.000000   0.000000
11     gradient_boosting   0.955100  0.003276    1.000000   0.000000
9    k-nearest_neighbors   0.954000  0.000835    0.977825   0.000655
6          random_forest   0.922550  0.004791    0.952575   0.001124
5            extra_trees   0.914250  0.004660    0.939175   0.000373
0             perceptron   0.867700  0.001187    0.891025   0.004880
1         sgd_classifier   0.865951  0.008922    0.900500   0.003892
3             linear_svc   0.861100  0.004611    0.937500   0.002822
10   logistic_regression   0.857900  0.001247    0.974175   0.002991
4          decision_tree   0.820500  0.002878    1.000000   0.000000
8              ada_boost   0.680353  0.052878    0.691624   0.047616
2   gaussian_naive bayes   0.561000  0.009317    0.567750   0.002612


In [152]:
# learn, predict and report
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.externals.joblib import Parallel, delayed, dump
test_id = np.arange(1, test_num+1)
results = {}
def process_one_model(name, model):
    model.fit(train_x, train_y)
    predict_y = model.predict(train_x)
    output = model.predict(test_x)
    results[name] = output
    submit = pd.DataFrame(data={'ImageId':test_id, 'Label':output})
    submit.to_csv('{}_submit.csv'.format(name), index=False)
    #dump(model, '{}.learn'.format(name))
    print(name)
    print(confusion_matrix(train_y, predict_y))
    print(classification_report(train_y, predict_y))
Parallel(n_jobs=n_cpus)(delayed(process_one_model)(name, model) for name,model in models)

[None, None, None, None, None, None, None, None, None, None, None, None]

             precision    recall  f1-score   support

          0       1.00      1.00      1.00      1932
          1       1.00      1.00      1.00      2266
          2       1.00      1.00      1.00      2010
          3       1.00      1.00      1.00      2080
          4       1.00      1.00      1.00      1971
          5       1.00      1.00      1.00      1794
          6       1.00      1.00      1.00      1982
          7       1.00      1.00      1.00      2070
          8       1.00      1.00      1.00      1915
          9       1.00      1.00      1.00      1980

avg / total       1.00      1.00      1.00     20000


[[1932    0    0    0    0    0    0    0    0    0]
 [   0 2266    0    0    0    0    0    0    0    0]
 [   0    0 2010    0    0    0    0    0    0    0]
 [   0    0    0 2080    0    0    0    0    0    0]
 [   0    0    0    0 1971    0    0    0    0    0]
 [   0    0    0    0    0 1794    0    0    0    0]
 [   0    0    0    0    0    0 1982    0    0    0]
 [   0    0    0    0    0    0    0 2070    0    0]
 [   0    0    0    0    0    0    0    0 1915    0]
 [   0    0    0    0    0    0    0    0    0 1980]]

gradient_boosting


             precision    recall  f1-score   support

          0       0.98      1.00      0.99      1932
          1       0.97      1.00      0.98      2266
          2       0.99      0.97      0.98      2010
          3       0.97      0.98      0.98      2080
          4       0.99      0.98      0.98      1971
          5       0.98      0.97      0.98      1794
          6       0.99      0.99      0.99      1982
          7       0.98      0.98      0.98      2070
          8       0.99      0.95      0.97      1915
          9       0.98      0.98      0.98      1980

avg / total       0.98      0.98      0.98     20000


...

[[1894    0    1    0    2    8    3    6   14    4]
 [   0 2090   11    7    0    4    5   28  112    9]
 [   5    3 1718   59   26    5   15   59  110   10]
 [   2    0   24 1819    2   13    4   39  154   23]
 [   0    3    2    6 1772    3    8   45   38   94]
 [  17    1    5  129    9 1182   31   27  338   55]
 [  11    1    5    4    5   22 1899    8   27    0]
 [   0    0    7    3    2    0    0 2021    6   31]
 [   9    3    3   36    1   19    3   33 1771   37]
 [   4    0    1   17   26    3    0  228   74 1627]]

sgd_classifier


             precision    recall  f1-score   support

          0       0.93      0.98      0.95      1932
          1       0.96      0.95      0.95      2266
          2       0.87      0.88      0.87      2010
          3       0.95      0.75      0.84      2080
          4       0.85      0.92      0.88      1971
          5       0.98      0.57      0.72      1794
          6       0.93      0.96      0.94      1982
          7       0.74      0.96      0.84      2070
          8       0.58      0.94      0.71      1915
          9       0.94      0.49      0.64      1980

avg / total       0.87      0.84      0.84     20000


[[1888    1    8    3    2    3   11    2   13    1]
 [   0 2146   12    2    2    0    7   54   43    0]
 [  17   13 1766    4   31    2   28   44  104    1]
 [  23    8  151 1555    9    9   14   48  257    6]
 [   4    7    5    0 1807    0    5   31   98   14]
 [  45   16   46   64   66 1016   78   36  403   24]
 [  17    6   19    0   16    1 1904    5   14    0]
 [   7    3   16    2   14    0    1 1984   26   17]
 [  18   31   13   11    9    5   10   22 1793    3]
 [  16   10    4    2  161    1    0  453  360  973]]

perceptron


In [153]:
# TODO bagging
# from sklearn.ensemble import BaggingClassifier

In [154]:
# TODO stacking
# from mlxtend.classifier import StackingClassifier

In [155]:
!head svm_submit.csv

ImageId,Label
1,2
2,0
3,9
4,9
5,3
6,7
7,0
8,3
9,0
